# **FullMap Cleaning**

Es important tenir les dades geografiques correctament normalitzades per poder relacionar tots els datasets de manera correcta ja sigui per municipi, districte, barri, o secció censal.

Aquest codi neteja el dataset FullMap (full_map.csv o full_map.geojson).

In [4]:
# IMPORTS

import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import numpy as np
import pandas as pd
import seaborn as sns
from shapely import wkt
import json
import datetime
import csv
import pandas as pd

In [5]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
ROOT = "/content/drive/Shareddrives/GESTIÓ DE PROJECTES/X. Entregable/"
SECCIONS_CENSALS_SHP = ROOT + "cleaning/raw_data/España_Seccionado2023_ETRS89H30/"
BARRIS_CSV = ROOT + 'cleaning/raw_data/BarcelonaCiutat_SeccionsCensals.csv'
CLEAN_DATA_PATH = ROOT + "cleaning/clean_data/"

## Mapa seccions censals INE

* **Font**: INE
* **URL**: https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259952026632&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout

Aquest dataset detalla totes les seccions censals d'Espanya amb informació bàsica com la CCAA, la provincia, o el municipi al que pertany cadascuna. A més, inclou informació GEO amb el polygon de la secció.

Es filtra només la provincia de Barcelona i es crea una columna True/False que indica si la secció està dins d'algún dels municipis on opera Aigües de Barcelona.

Aquest dataset no conté informació de barris ja que aquesta és una divisió territorial única de la ciutat de Barcelona.

In [17]:
# MAPA SECCIONS CENSALS INE

# Import dataset
map_df = gpd.read_file(SECCIONS_CENSALS_SHP, where="NPRO='Barcelona'")

# Convert coordinate projection
map_df = map_df.to_crs(epsg=4326)

# Highlight rows where AB operates
OPERATED_BY_AB_LIST = ["Barcelona", "Viladecans", "Gavà", "Sant Feliu de Llobregat",
                           "Hospitalet de Llobregat, L'", "Sant Adrià de Besòs"]
def operated_by_AB(row):
    return row['NMUN'] in OPERATED_BY_AB_LIST

map_df["AB_Region"] = map_df.apply(operated_by_AB, axis=1)
map_df = map_df.drop(columns=["CNUT1", "CNUT2", "CNUT3", "CLAU2"])

# Show data format
map_df.head(3)

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,NPRO,NCA,CNUT0,NMUN,geometry,AB_Region
0,0800101001,08001,001,01,001,08,09,0800101,Barcelona,Cataluña,ES,Abrera,"POLYGON ((1.90971 41.50175, 1.90878 41.50195, ...",False
1,0800101002,08001,002,01,001,08,09,0800101,Barcelona,Cataluña,ES,Abrera,"POLYGON ((1.90357 41.52143, 1.90370 41.52039, ...",False
2,0800101003,08001,003,01,001,08,09,0800101,Barcelona,Cataluña,ES,Abrera,"POLYGON ((1.92482 41.49562, 1.92474 41.49565, ...",False


## Barris BCN

Aquest dataset ens permet relacionar seccions censals i barris de Barcelona.

* Font: Open Data BCN
* URL: https://opendata-ajuntament.barcelona.cat/data/es/dataset/20170706-districtes-barris/resource/e16856a7-b3c0-4c32-a468-cc190cbbf7a9

In [18]:
# MAPA BARRIS BCN

# Import dataset
barris = pd.read_csv(BARRIS_CSV).drop(columns=["geometria_etrs89"])
barris["geometria_wgs84"] = barris["geometria_wgs84"].apply(wkt.loads)
barris = gpd.GeoDataFrame(barris, geometry=barris["geometria_wgs84"], crs="4326")
barris = barris.drop(columns=["geometria_wgs84", "codi_aeb", "geometry"])

# Add paddings to codes to match the INE dataset
barris['codi_districte'] = barris['codi_districte'].astype(str).str.zfill(2)
barris['codi_barri'] = barris['codi_barri'].astype(str).str.zfill(2)
barris['codi_seccio_censal'] = barris['codi_seccio_censal'].astype(str).str.zfill(3)

# Write complete CUSEC
INE_CODI_PROVINCIA_BARCELONA = "08"
INE_CODI_MUNICIPI_BARCELONA = "019"
barris["CUSEC"] = INE_CODI_PROVINCIA_BARCELONA + INE_CODI_MUNICIPI_BARCELONA + barris['codi_districte'] + barris['codi_seccio_censal']

# Column drops and renames
barris = barris.drop(columns=["codi_districte", "codi_seccio_censal"])
barris = barris.rename(columns={"nom_districte":"NDIS", "codi_barri":"CBAR", "nom_barri":"NBAR"})

# Show data format
barris.head(3)

,NDIS,CBAR,NBAR,CUSEC
0,Ciutat Vella,01,el Raval,0801901001
1,Ciutat Vella,01,el Raval,0801901002
2,Ciutat Vella,01,el Raval,0801901003


## Full map

Finalment es junten tots els datasets i es crea una taula única amb totes les subdivisions territorials possibles.

In [19]:
full_map = map_df.merge(barris, on='CUSEC', how='left')

full_map['NDIS'].fillna(full_map['NMUN']+"_"+full_map["CDIS"], inplace=True)
full_map['CBAR'].fillna(full_map["CDIS"], inplace=True)
full_map['NBAR'].fillna(full_map['NMUN']+"_"+full_map["CDIS"], inplace=True)
full_map["CUBAR"] = full_map["CPRO"] + full_map["CMUN"] + full_map["CDIS"] + full_map["CBAR"]

full_map = full_map.drop(columns=["CPRO", "CCA"])

full_map.head(3)

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CUDIS,NPRO,NCA,CNUT0,NMUN,geometry,AB_Region,NDIS,CBAR,NBAR,CUBAR
0,0800101001,08001,001,01,001,0800101,Barcelona,Cataluña,ES,Abrera,"POLYGON ((1.90971 41.50175, 1.90878 41.50195, ...",False,Abrera_01,01,Abrera_01,080010101
1,0800101002,08001,002,01,001,0800101,Barcelona,Cataluña,ES,Abrera,"POLYGON ((1.90357 41.52143, 1.90370 41.52039, ...",False,Abrera_01,01,Abrera_01,080010101
2,0800101003,08001,003,01,001,0800101,Barcelona,Cataluña,ES,Abrera,"POLYGON ((1.92482 41.49562, 1.92474 41.49565, ...",False,Abrera_01,01,Abrera_01,080010101


## **Export**

Exporting the full_map dataset to /data_clean/

In [20]:
FULL_MAP_GEOJSON_OUTPUT_FILE = "full_map.geojson"
full_map[full_map["AB_Region"]==True].to_file(CLEAN_DATA_PATH+FULL_MAP_GEOJSON_OUTPUT_FILE, driver="GeoJSON")

In [21]:
FULL_MAP_DATA_OUTPUT_FILE = "full_map.csv"
full_map[full_map["AB_Region"]==True].to_csv(CLEAN_DATA_PATH+FULL_MAP_DATA_OUTPUT_FILE, index=False)

# **Resultat final**

La taula resultant conté les següents columnes:

* País
    * CNUT0: ES
* Comunitat autónoma:
    * CCA: Codi Comunitat Aut.
    * NCA: Nom Comunitat Aut.
* Provincia:
    * CPRO: Codi provincia
    * NPRO: Nom provincia
* Municipi:
    * CUMUN: Codi únic del municipi (CPRO+CMUN)
    * CMUN: Codi del municipi dins la seva provincia
    * NMUN: Nom del municipi
* Districte:
    * CUDIS: Codi únic del districte (CPRO+CMUN+CDIS)
    * CDIS: Codi del districte dins el seu municipi
    * NDIS: Nom del districte
* Barri:
  (només estàn disponibles per Barcelona, per la resta de municipis, el barri es igual al districte)
    * CUBAR: Codi únic del barri (CPRO+CMUN+CDIS+CBAR)
    * CBAR: Codi del barri dins el seu municipi
    * NBAR: Nom del barri
    
* Secció censal (as of 2023):
    * CUSEC: Codi únic de la secció censal (CPRO+CMUN+CDIS+CSEC)
    * CSEC: Codi de la secció censal dins del seu districte

* Está gestionat per Aigües de Barcelona?
    * AB_Region: True/False
* Geographical data
    * geometry: coordenades del polígon de la secció censal
    
Aquest dataset ens permet relacionar municipis-districties-barris-seccions censals i, per tant, utilitzar qualsevol dataset independentment de si està agregat per municipi, districte, barri, o secció censal. A més, les dades geometriques de coordenades ens permeten visualitzar les dades en forma de mapa a Python (o a Power BI, o QGis).

## Import template

Template to import and use the full_map dataset as a dataframe of geoDataframe.

In [22]:
from shapely import wkt
import pandas as pd
import geopandas as gpd

In [26]:
# TO USE IT JUST AS DataFrame...
full_map = pd.read_csv(CLEAN_DATA_PATH+FULL_MAP_DATA_OUTPUT_FILE, dtype={"CUSEC":str, "CUMUN":str, "CSEC":str, "CDIS":str, "CMUN":str, "CUDIS":str, "CUBAR":str, "CBAR":str})

In [24]:
# TO USE IT JUST AS GeoDataFrame...
full_map = pd.read_csv(CLEAN_DATA_PATH+FULL_MAP_DATA_OUTPUT_FILE, dtype={"CUSEC":str, "CUMUN":str, "CSEC":str, "CDIS":str, "CMUN":str, "CUDIS":str, "CUBAR":str, "CBAR":str})
full_map["geometry"] = gpd.GeoSeries.from_wkt(full_map["geometry"])
full_map = gpd.GeoDataFrame(full_map, geometry="geometry", crs='epsg:4326')